In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
import pyvista as pv
from matplotlib.colors import ListedColormap
import time
import vtk

In [2]:
# Load in data
earthquake_data = pv.read('cleaned_earthquake_data.vtk')
earthquake_data

PolyData (0x2bc0b0e6be0)
  N Cells:    1
  N Points:   31500
  N Strips:   0
  X Bounds:   -1.272e+07, -1.211e+07
  Y Bounds:   4.404e+06, 5.236e+06
  Z Bounds:   0.000e+00, 6.628e+00
  N Arrays:   31

In [3]:
# Get years without duplicates
active_years = []
for year in earthquake_data.point_data['Year']:
    if year not in active_years:
        active_years.append(year)
active_years.sort()

In [4]:
# Define hardcoded boundaries for quadrants
regional_verts = np.array([
    [[-111.55, 42.00, 0.0], [-111.55, 37.00, 0.0]],
    [[-114.05, 39.50, 0.0], [-109.05, 39.50, 0.0]]
])


# Define hardcoded boundaries for Utah border
utah_corners = np_points = np.array([
    [-114.05, 42.00, 0.0], 
    [-111.05, 42.00, 0.0],
    [-111.05, 41.00, 0.0],
    [-109.05, 41.00, 0.0],
    [-109.05, 37.00, 0.0],
    [-114.05, 37.00, 0.0]
])

In [21]:
# Get info to build meshes by year
def get_coords_and_mags_by_year(year):
    lat_lng = []
    mags = []
    
    indices = np.where(earthquake_data.point_data['Year'] == year)[0]
    for i in indices:
        x, y, z = float(earthquake_data.point_data['Long'][i]), earthquake_data.point_data['Lat'][i], 0
        lat_lng.append((x, y, z))
        mags.append(earthquake_data.point_data['Mag'][i])
        
    lat_lng_np = np.array(lat_lng)   
    mags_np = np.array(mags)
    
    return lat_lng_np, mags_np.flatten()

def get_damage_per_mag(mag):
    if mag <= 2.5:
        return 0
    elif mag <= 5.4:
        return 1
    elif mag <= 6:
        return 2
    elif mag <= 6.9:
        return 3
    elif mag <= 7.9:
        return 4
    else:
        return 5

def get_aggregate_data_by_year(year):
    # Get aggregate data for region
    indices = np.where(earthquake_data.point_data['Year'] <= year)[0]
    quake_count = {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}
    total_damage = {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}
    mags = {'Q1': [], 'Q2': [], 'Q3': [], 'Q4': []}
    for i in indices:
        x, y = float(earthquake_data.point_data['Long'][i]), float(earthquake_data.point_data['Lat'][i])
        if x < 111.55 and y > 39.5:
            # Q1
            quake_count['Q1'] += 1
            mag = earthquake_data.point_data['Mag'][i]
            total_damage['Q1'] += get_damage_per_mag(mag)
            mags['Q1'].append(mag)
        elif x > 111.55 and y > 39.5:
            # Q2
            quake_count['Q2'] += 1
            mag = earthquake_data.point_data['Mag'][i]
            total_damage['Q2'] += get_damage_per_mag(mag)
            mags['Q2'].append(mag)
        elif x > 111.55 and y < 39.5:
            # Q3
            quake_count['Q3'] += 1
            mag = earthquake_data.point_data['Mag'][i]
            total_damage['Q3'] += get_damage_per_mag(mag)
            mags['Q3'].append(mag)
        else:
            # Q4
            quake_count['Q4'] += 1
            mag = earthquake_data.point_data['Mag'][i]
            total_damage['Q4'] += get_damage_per_mag(mag)
            mags['Q4'].append(mag)
            
    for quadrant in mags:
        mags_np = np.array(mags[quadrant])
        if mags_np.any():
            avg_mag = np.mean(mags_np)
            max_mag = mags_np.max()
            mags[quadrant] = [avg_mag, max_mag]
        else:
            mags[quadrant] = [0,0]

    return total_damage, mags, quake_count

In [22]:
get_aggregate_data_by_year(1900)

({'Q1': 44, 'Q2': 30, 'Q3': 0, 'Q4': 0},
 {'Q1': [3.450705674418604, 5.577],
  'Q2': [3.513900000000001, 4.813],
  'Q3': [0, 0],
  'Q4': [0, 0]},
 {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0})

In [7]:
# Set up custom cmap, note that we don't have mags that hit the red/green regions, but still display them in our classifications
c1 = np.array([0.99, 0.0, 0.0]) # red
c2 = np.array([0.0, 0.99, 0.0]) # green
c3 = np.array([0.0, 0.0, 0.99]) # blue
c4 = np.array([0.99, 0.99, 0.0]) # yellow
c5 = np.array([0.99, 0.0, 0.99]) # pink
c6 = np.array([0.0, 0.99, 0.99]) # cyan

mapping = np.linspace(earthquake_data.point_data['Mag'].min(), earthquake_data.point_data['Mag'].max(), 256)
new_colors = np.empty((256, 3))
new_colors[mapping >= 8.0] = c1
new_colors[mapping < 8] = c2
new_colors[mapping < 7] = c3
new_colors[mapping < 6.1] = c4
new_colors[mapping < 5.5] = c5
new_colors[mapping < 2.5] = c6

mag_classification_cmap = ListedColormap(new_colors)
classes_lut = pv.LookupTable(cmap=mag_classification_cmap)


In [8]:
# Set up legend based on classifications
legend_entries = []
legend_entries.append(['Usually not felt', c6])
legend_entries.append(['Minor damage', c5])
legend_entries.append(['Slight damage', c4])
legend_entries.append(['Lots of damage', c3])
legend_entries.append(['Serious damage', c2])
legend_entries.append(['Widespread devastation', c1])

In [ ]:
# Define hardcoded mesh and mesh points for aggregate data display
tot_damage_mesh = pv.UnstructuredGrid()
tot_damage_mesh.points = np.array([
    [-112.7, 40.4, 0], 
    [-112.7, 38.15, 0], 
    [-110.2, 40.4, 0], 
    [-110.2, 38.15, 0]
])

avg_mag_mesh = pv.UnstructuredGrid()
avg_mag_mesh.points = np.array([
    [-112.9, 40.6, 0], 
    [-112.9, 38.35, 0],
    [-110.4, 40.6, 0], 
    [-110.4, 38.35, 0]
])

max_mag_mesh = pv.UnstructuredGrid()
max_mag_mesh.points = np.array([
    [-112.7, 40.6, 0], 
    [-112.7, 38.35, 0], 
    [-110.2, 40.6, 0], 
    [-110.2, 38.35, 0]
])

quake_count_mesh = pv.UnstructuredGrid()
quake_count_mesh.points = np.array([
    [-112.9, 40.4, 0],
    [-112.9, 38.15, 0],
    [-110.4, 40.4, 0],
    [-110.4, 38.15, 0],
])
#     context_mesh.points = np.array([
#         [-112.8, 40.5, 0], [-112.7, 40.4, 0], [-112.9, 40.6, 0], [-112.7, 40.6, 0], [-112.9, 40.4, 0],
#         [-112.8, 38.25, 0], [-112.7, 38.15, 0], [-112.9, 38.35, 0], [-112.7, 38.35, 0], [-112.9, 38.15, 0],
#         [-110.3, 40.5, 0], [-110.2, 40.4, 0], [-110.4, 40.6, 0], [-110.2, 40.6, 0], [-110.4, 40.4, 0],
#         [-110.3, 38.25, 0], [-110.2, 38.15, 0], [-110.4, 38.35, 0], [-110.2, 38.35, 0], [-110.4, 38.15, 0],
#     ])

In [9]:
# Creates mesh by year and stores coords and mag glyphs in global variables
def create_mag_mesh(year, mode):
    # Create mesh with coords and mag scalars
    mag_mesh = pv.UnstructuredGrid()
    mag_mesh.points, mag_mesh.point_data["Mag"] = get_coords_and_mags_by_year(year)
    
    # Grab aggregate data
    tot_damage, avg_mag, max_mag, quake_count = get_aggregate_data_by_year(year)
    
    # Select color map
    mode_cmap = ''
    if mode == 'classes':
        mode_cmap = classes_lut
    else:
        mode_cmap = 'seismic'
    
    # Build coord mesh and mag glyphs
    pl.subplot(0, 0)
    coords = pl.add_mesh(mag_mesh, point_size=1)
    ball = pv.Sphere(radius=0.05, theta_resolution=35, phi_resolution=35)
    mag_glyphs = mag_mesh.glyph(geom=ball, orient=False, scale="Mag")
    mags = pl.add_mesh(mag_glyphs, cmap=mode_cmap)
    
    # Context stuff
    pl.subplot(0, 1)
    con_coords = pl.add_mesh(context_mesh, point_size=1)
    context_glyphs = context_mesh.glyph(geom=ball, orient=False)
    context = pl.add_mesh(context_glyphs, cmap=mode_cmap)
    
    # Set visibility to false, will be toggled interactively later
    coords.visibility = False
    mags.visibility = False
    
    # Store meshes by year
    coord_meshes[year] = coords
    mag_meshes[year] = mags
    context_meshes[year] = context
    return

# Only the selected year has visible coords/mags
def move_year_forward():
    # Must be global, won't render otherwise
    global year_index
    year_index += 1
    if year_index > len(active_years) - 1:
        year_index -= 1
        return
    else:
        year = active_years[year_index]
        coord_meshes[year].visibility = True
        mag_meshes[year].visibility = True
        if not year_index == 0:
            prior_year = active_years[year_index - 1]
            coord_meshes[prior_year].visibility = False
            mag_meshes[prior_year].visibility = False

        pl.subplot(0, 0)
        pl.add_text("Year: "+ str(year) + "\n f: next year\nb: prior year", "lower_left", name='year_text', font_size=12)
        pl.update()

def move_year_backward():
    global year_index    
    year_index -= 1
    if year_index < 0:
        year_index += 1
        return
    else:
        year = active_years[year_index]
        coord_meshes[year].visibility = True
        mag_meshes[year].visibility = True
        if not year_index == len(active_years) - 1:
            prior_year = active_years[year_index + 1]
            coord_meshes[prior_year].visibility = False
            mag_meshes[prior_year].visibility = False
        pl.subplot(0, 0)
        pl.add_text("Year: "+ str(year) + "\n f: next year\nb: prior year", "lower_left", name='year_text', font_size=12)
        pl.update()

In [10]:
# Mode refers to color map, either use 'classes' for custom classification, otherwise it will use 'seismic' (blue to red)
def run_vis(mode='classes'):
    global year_text
    # Display approximate boundaries for Utah
    pl.subplot(0, 0)
    outline_mesh = pv.PolyData(utah_corners)
    pl.add_mesh(outline_mesh, point_size=15)
    poly_line = pv.MultipleLines(points=utah_corners)
    pl.add_mesh(poly_line, line_width=2)
    line = pv.Line(pointa=utah_corners[0], pointb=utah_corners[5])
    pl.add_mesh(line, line_width=2, color='white')
    
    # Setup regional display
    vert_reg_line = pv.Line(pointa=regional_verts[0][0], pointb=regional_verts[0][1])
    pl.add_mesh(vert_reg_line, line_width=2, color='white')
    hor_reg_line = pv.Line(pointa=regional_verts[1][0], pointb=regional_verts[1][1])
    pl.add_mesh(hor_reg_line, line_width=2, color='white')
    
    # Load points for all years, but we only display one at a time
    for year in active_years:
        create_mag_mesh(year, mode)
        
    pl.add_title("Utah Earthquakes 1850-2016", font_size=14)
    pl.add_text("Year: \n f: next year\nb: prior year", "lower_left", name='year_text', font_size=12)
        
    # Interactive events to move through years
    pl.add_key_event("f", move_year_forward)
    pl.add_key_event("b", move_year_backward)
    pl.view_xy()
    
    # Setup context visualization
    pl.subplot(0, 1)
    pl.add_text("Context Visualization")
    
    pl.add_mesh(outline_mesh, point_size=15)
    pl.add_mesh(poly_line, line_width=2)
    pl.add_mesh(line, line_width=2, color='white')
    
    # Setup regional display
    pl.add_mesh(vert_reg_line, line_width=2, color='white')
    pl.add_mesh(hor_reg_line, line_width=2, color='white')
    
    pl.view_xy()
    pl.link_views()
    pl.show(cpos='xy')

In [11]:
# Values used globally, need to reset each run
year_index = -1
coord_meshes = {}
mag_meshes = {}
context_meshes = {}

pl = pv.Plotter(notebook=False, shape=(1, 2))
run_vis()

C:\Users\alexa\anaconda3\lib\site-packages\pyvista\core\filters\data_set.py:2108: UserWarning: No data to use for scale. scale will be set to False.
  warnings.warn("No data to use for scale. scale will be set to False.")
